In [1]:
from iuvs import io
import pandas as pd
l1bfiles = io.l1b_filenames()

In [2]:
from IPython.parallel import Client
rc = Client()

In [3]:
dview = rc.direct_view()
lview = rc.load_balanced_view()

In [34]:
def get_header_df(header, drop_comment=True):
    import pandas as pd
    df = pd.DataFrame(header.values(),
                      index=header.keys())
    return df.drop('COMMENT KERNEL'.split()) if drop_comment else df

def save_to_hdf(df, basename):
    import os
    HOME = os.environ['HOME']
    newfname = os.path.splitext(basename)[0] + '.h5'
    path = os.path.join(HOME, 'output', newfname)
    print(path)
    df.to_hdf(path, 'df', format='t')
    
def process_img_headers(fname):
    from iuvs import io
    import os
    from astropy.io import fits
    iuvsfname = io.Filename(fname)
    l1b = io.L1BReader(fname)
    hdu = l1b.hdulist[0]
    hdu.verify('fix')
    df = get_header_df(hdu.header).T.convert_objects()
    basename = os.path.basename(fname)
    df['filename'] = basename
    save_to_hdf(df, basename)
    return

In [31]:
dview.push({'get_header_df':get_header_df,
            'save_to_hdf': save_to_hdf})

<AsyncResult: finished>

In [35]:
results = lview.map_async(process_img_headers, l1bfiles)

In [40]:
import time
while not results.ready():
    print("{:.1f} %".format(100*results.progress/len(l1bfiles)))
    time.sleep(30)          

2.1 %
2.3 %
2.7 %
2.9 %
3.3 %
3.7 %
4.0 %
4.4 %
4.7 %
5.1 %
5.5 %
5.8 %
6.2 %
6.6 %
7.0 %
7.3 %
7.6 %
7.9 %
8.3 %
8.6 %
9.0 %
9.4 %
9.8 %
10.2 %
10.6 %
10.9 %
11.3 %
11.6 %
12.0 %
12.4 %
12.8 %
13.1 %
13.4 %
13.7 %
14.0 %
14.3 %
14.6 %
15.0 %
15.3 %
15.6 %
15.9 %
16.2 %
16.5 %
16.9 %
17.1 %
17.4 %
17.7 %
18.1 %
18.5 %
18.8 %
19.2 %
19.5 %
19.9 %
20.3 %
20.6 %
21.0 %
21.4 %
21.7 %
22.1 %
22.5 %
23.0 %
23.3 %
23.6 %
24.0 %
24.4 %
24.6 %
25.0 %
25.2 %
25.6 %
25.9 %
26.4 %
26.7 %
27.1 %
27.5 %
27.8 %
28.2 %
28.6 %
29.0 %
29.3 %
29.6 %
29.9 %
30.3 %
30.6 %
31.0 %
31.4 %
31.8 %
32.2 %
32.5 %
32.7 %
33.1 %
33.4 %
33.7 %
34.1 %
34.4 %
34.8 %
35.2 %
35.5 %
35.9 %
36.2 %
36.5 %
36.9 %
37.2 %
37.5 %
37.8 %
38.2 %
38.6 %
38.8 %
39.2 %
39.6 %
39.9 %
40.3 %
40.6 %
41.0 %
41.3 %
41.7 %
42.0 %
42.3 %
42.6 %
42.9 %
43.4 %
43.7 %
44.0 %
44.3 %
44.7 %
45.0 %
45.3 %
45.8 %
46.0 %
46.4 %
46.8 %
47.0 %
47.4 %
47.8 %
48.1 %
48.4 %
48.7 %
48.9 %
49.3 %
49.7 %
50.0 %
50.4 %
50.7 %
51.1 %
51.6 %
51.9 %
52.3 %
5

In [23]:
l1b = process_img_headers(l1bfiles[0])

In [24]:
hdu = l1b.hdulist[0]
hdu.verify('silentfix')

In [25]:
hdu.header

SIMPLE  =                    T /Primary Header created by MWRFITS v1.11         
BITPIX  =                  -32 /                                                
NAXIS   =                    2 /                                                
NAXIS1  =                  512 /                                                
NAXIS2  =                  512 /                                                
EXTEND  =                    T /Extensions may be present                       
COMMENT MAVEN IUVS Level 1B Data Product                                        
COMMENT IUVS Principal Investigator N. Schneider                                
COMMENT Laboratory for Atmospheric and Space Physics                            
COMMENT University of Colorado Boulder                                          
COMMENT 1234 Innovation Drive, Boulder CO 80303                                 
COMMENT MAVEN Mission scientific and model results are open to all.             
COMMENT Users should contact